In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from math import sqrt
from unidecode import unidecode
from yellowbrick.cluster import KElbowVisualizer
import re
import shutil#For study. Final version not necessary
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import math

In [3]:
print('Initializing virtual boot...')
url='https://buscacepinter.correios.com.br/app/cep/index.php'

#option=Options()
#option.add_argument("--headless")
#option.add_experimental_option("excludeSwitches", ["enable-automation"])
#option.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome()#options=option)
driver.maximize_window()
print('Done.')
time.sleep(1)
print('Acessing https://www.correios.com.br/... ')
driver.get(url)
print('Done.')

Initializing virtual boot...
Done.
Acessing https://www.correios.com.br/... 
Done.


In [124]:
#encontrar ceps
df_ita_correios=pd.DataFrame(columns=['Logradouro/Nome','Bairro/Distrito','Localidade/UF','CEP'])

for cep in range(18200,18220):
    print(f'Scraping table {cep}...')
    time.sleep(1)
    driver.find_element(By.XPATH,"//*[@id='cep']").send_keys(cep)
    time.sleep(1)
    driver.find_element(By.XPATH,"//*[@id='btn_pesquisar']").click()
    time.sleep(5)
    
    total_results=driver.find_element(By.XPATH,"//*[@id='navegacao-total']").text
    time.sleep(1)
    #find total of ocorrencies
    results=re.findall('[0-9]+',total_results)
    results_per_pages=int(results[1])
    total_results_number=int(results[2])
    print(f'Number of occurences: {total_results_number}')
    
    if results_per_pages==50:
        #Iqual 50 more tables loop for clicks
        tables_number=math.ceil(total_results_number/results_per_pages)
        clicks_number=(tables_number-1)
        print(f'The number of tables is: {tables_number}')
        n=0
        for df_table in range(clicks_number):
            #table soup
            element=driver.find_element(By.XPATH,"//*[@id='resultado-DNEC']")
            html_content = element.get_attribute('outerHTML')
            soup = BeautifulSoup(html_content,'html.parser')
            table = soup.find(name='table')
            df_table=pd.read_html(str(table))[0]
            df_ita_correios=df_ita_correios.append(df_table,ignore_index=True)
            print(f'Table {cep}-{n} Scrapped.')
            n=n+1
            driver.find_element(By.XPATH,"//*[@id='navegacao-resultado']/a[2]").click()
            time.sleep(5)
        #Last Soup
        element=driver.find_element(By.XPATH,"//*[@id='resultado-DNEC']")
        html_content = element.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content,'html.parser')
        table = soup.find(name='table')
        df_table=pd.read_html(str(table))[0]
        df_ita_correios=df_ita_correios.append(df_table,ignore_index=True)
        print(f'Table {cep}-{n} Scrapped.')
        n=n+1
    else:
        n=0
        #Less than 50 ocurrences just one scrap
        #table soup
        print('The number of tables is: 0')
        element=driver.find_element(By.XPATH,"//*[@id='resultado-DNEC']")#find table
        html_content = element.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content,'html.parser')
        table = soup.find(name='table')
        df_table=pd.read_html(str(table))[0]
        df_ita_correios=df_ita_correios.append(df_table,ignore_index=True) 
        print(f'Table {cep}-{n} Scrapped.')
        n=n+1
        time.sleep(5)
    element=driver.find_element(By.XPATH,"//*[@id='btn_voltar']").click()
    time.sleep(2)

Scraping table 18200...
Number of occurences: 120
The number of tables is: 3
Table 18200-0 Scrapped.
Table 18200-1 Scrapped.
Table 18200-2 Scrapped.
Scraping table 18201...
Number of occurences: 78
The number of tables is: 2
Table 18201-0 Scrapped.
Table 18201-1 Scrapped.
Scraping table 18202...
Number of occurences: 68
The number of tables is: 2
Table 18202-0 Scrapped.
Table 18202-1 Scrapped.
Scraping table 18203...
Number of occurences: 67
The number of tables is: 2
Table 18203-0 Scrapped.
Table 18203-1 Scrapped.
Scraping table 18204...
Number of occurences: 19
The number of clicks is: 0
Table 18204-0 Scrapped.
Scraping table 18205...
Number of occurences: 85
The number of tables is: 2
Table 18205-0 Scrapped.
Table 18205-1 Scrapped.
Scraping table 18206...
Number of occurences: 124
The number of tables is: 3
Table 18206-0 Scrapped.
Table 18206-1 Scrapped.
Table 18206-2 Scrapped.
Scraping table 18207...
Number of occurences: 189
The number of tables is: 4
Table 18207-0 Scrapped.
Table

In [115]:
for a in range(2):
    driver.find_element(By.XPATH,"//*[@id='navegacao-resultado']/a[2]").click()
    print(a)
    time.sleep(10)

0
1


In [125]:
df_ita_correios

,Logradouro/Nome,Bairro/Distrito,Localidade/UF,CEP
0,Rua Saldanha Marinho,Centro,Itapetininga/SP,18200-001
1,Praça da Bandeira,Centro,Itapetininga/SP,18200-002
2,Praça Duque de Caxias,Centro,Itapetininga/SP,18200-003
3,Rua Campos Salles,Centro,Itapetininga/SP,18200-005
4,Praça Marechal Deodoro da Fonseca,Centro,Itapetininga/SP,18200-007
...,...,...,...,...
1671,"Rua Elias Jorge Daniel, nº 56 - Distrito de Gr...",NaN,Gramadinho/SP,18218-990
1672,Rua Elias Jorge Daniel 56AGC Gramadinho,Centro/Gramadinho,Itapetininga/SP,18218-970
1673,Rua Onório Sales s/nAGC Varginha,Centro/Varginha,Itapetininga/SP,18218-971
1674,NaN,Morro do Alto,Itapetininga/SP,18219-000


In [46]:
results

['1', '50', '149']

In [36]:
df_ita_correios

,Logradouro/Nome,Bairro/Distrito,Localidade/UF,CEP
0,Rua Saldanha Marinho,Centro,Itapetininga/SP,18200-001
1,Praça da Bandeira,Centro,Itapetininga/SP,18200-002
2,Praça Duque de Caxias,Centro,Itapetininga/SP,18200-003
3,Rua Campos Salles,Centro,Itapetininga/SP,18200-005
4,Praça Marechal Deodoro da Fonseca,Centro,Itapetininga/SP,18200-007
...,...,...,...,...
695,Rua Eliziario Martins de Mello,Vila Regina,Itapetininga/SP,18209-310
696,Rua Maurício Jacinto de Almeida,Vila Regina,Itapetininga/SP,18209-320
697,Rua Arnaldo Soares Hungria,Vila Regina,Itapetininga/SP,18209-330
698,Rua Cassimiro Cardoso,Vila Regina,Itapetininga/SP,18209-340


In [16]:
test=pd.read_csv("bel coordenates.csv")

In [26]:
test=test.drop('Unnamed: 0',axis=1)

In [27]:
test_shuffle=test.sample(frac=1).reset_index(drop=True)

In [28]:
test

,address_components,formatted_address,place_id,types,geometry.bounds.northeast.lat,geometry.bounds.northeast.lng,geometry.bounds.southwest.lat,geometry.bounds.southwest.lng,geometry.location.lat,geometry.location.lng,geometry.location_type,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,partial_match
0,"[{'long_name': 'Aeroporto', 'short_name': 'Aer...","Aeroporto, Belém - PA, Brazil",ChIJffGZlDZrpJIROa2KpIvpy-0,"['political', 'sublocality', 'sublocality_leve...",-1.135819,-48.450568,-1.146009,-48.459296,-1.141777,-48.455427,APPROXIMATE,-1.135819,-48.450568,-1.146009,-48.459296,NaN
1,"[{'long_name': 'Água Boa', 'short_name': 'Água...","Água Boa, Belém - PA, Brazil",ChIJl_gD0PJopJIR6ECQpt0XpD4,"['political', 'sublocality', 'sublocality_leve...",-1.218171,-48.432478,-1.264002,-48.473276,-1.250924,-48.453742,APPROXIMATE,-1.218171,-48.432478,-1.264002,-48.473276,True
2,"[{'long_name': 'Águas Lindas', 'short_name': '...","Águas Lindas, Belém - PA, Brazil",ChIJE-QDqs70pJIRf7CVFNuekTI,"['political', 'sublocality', 'sublocality_leve...",-1.400014,-48.390563,-1.411405,-48.397910,-1.405529,-48.395076,APPROXIMATE,-1.400014,-48.390563,-1.411405,-48.397910,True
3,"[{'long_name': 'Águas Negras (Icoaraci)', 'sho...","Águas Negras (Icoaraci), Belém - PA, 66820-180...",ChIJ-_k1nMZmpJIRv0mn7ZSRQN8,"['political', 'sublocality', 'sublocality_leve...",NaN,NaN,NaN,NaN,-1.303088,-48.453916,APPROXIMATE,-1.291933,-48.437908,-1.314243,-48.469923,NaN
4,"[{'long_name': 'Agulha (Icoaraci)', 'short_nam...","Agulha (Icoaraci), Belém - PA, 66811-033, Brazil",ChIJufLAiKhmpJIRE8_2LpkEBlI,"['political', 'sublocality', 'sublocality_leve...",NaN,NaN,NaN,NaN,-1.307735,-48.468183,APPROXIMATE,-1.296580,-48.452175,-1.318891,-48.484190,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,"[{'long_name': 'Val de Caes', 'short_name': 'V...","Val de Caes, Belém - PA, Brazil",ChIJiagB7xiKpJIRNC0siK474Xc,"['political', 'sublocality', 'sublocality_leve...",-1.367814,-48.454743,-1.403103,-48.495722,-1.389877,-48.473728,APPROXIMATE,-1.367814,-48.454743,-1.403103,-48.495722,NaN
72,"[{'long_name': 'Belém', 'short_name': 'Belém',...","Belém - State of Pará, Brazil",ChIJX26Y9WlmpJIRzImxai3bNuM,"['administrative_area_level_2', 'political']",-1.019427,-48.296124,-1.526585,-48.624408,-1.455729,-48.490178,APPROXIMATE,-1.019427,-48.296124,-1.526585,-48.624408,NaN
73,"[{'long_name': 'Vila', 'short_name': 'Vila', '...","Vila, Belém - PA, Brazil",ChIJk1VzIXprpJIR9mj26j6GZ04,"['political', 'sublocality', 'sublocality_leve...",-1.152420,-48.465938,-1.174713,-48.474167,-1.163874,-48.470122,APPROXIMATE,-1.152420,-48.465938,-1.174713,-48.474167,NaN
74,"[{'long_name': 'Vila (Mosqueiro)', 'short_name...","Vila (Mosqueiro), Belém - PA, 68447-000, Brazil",ChIJDUpBd-eLpJIRhivPgkEoCCY,"['political', 'sublocality', 'sublocality_leve...",-1.412450,-48.447034,-1.418568,-48.452324,-1.416307,-48.449371,APPROXIMATE,-1.412450,-48.447034,-1.418568,-48.452324,NaN


In [29]:
test_shuffle

,address_components,formatted_address,place_id,types,geometry.bounds.northeast.lat,geometry.bounds.northeast.lng,geometry.bounds.southwest.lat,geometry.bounds.southwest.lng,geometry.location.lat,geometry.location.lng,geometry.location_type,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,partial_match
0,"[{'long_name': 'Parque Guajará', 'short_name':...","Parque Guajará, Belém - PA, Brazil",ChIJlyRCTEdhpJIRomHIY4_zfvc,"['political', 'sublocality', 'sublocality_leve...",-1.306425,-48.449022,-1.342703,-48.469616,-1.322087,-48.461236,APPROXIMATE,-1.306425,-48.449022,-1.342703,-48.469616,NaN
1,"[{'long_name': 'Praia Grande', 'short_name': '...","Praia Grande, Belém - PA, Brazil",ChIJiQDW8kRrpJIRwWet6iTB7EE,"['political', 'sublocality', 'sublocality_leve...",-1.142454,-48.461256,-1.152607,-48.471670,-1.148082,-48.466409,APPROXIMATE,-1.142454,-48.461256,-1.152607,-48.471670,NaN
2,"[{'long_name': 'Marco', 'short_name': 'Marco',...","Marco, Belém - PA, Brazil",ChIJ96G9wAyMpJIR6gRyD01bam0,"['political', 'sublocality', 'sublocality_leve...",-1.420521,-48.447172,-1.448307,-48.472286,-1.435312,-48.461236,APPROXIMATE,-1.420521,-48.447172,-1.448307,-48.472286,NaN
3,"[{'long_name': 'Vila', 'short_name': 'Vila', '...","Vila, Belém - PA, Brazil",ChIJk1VzIXprpJIR9mj26j6GZ04,"['political', 'sublocality', 'sublocality_leve...",-1.152420,-48.465938,-1.174713,-48.474167,-1.163874,-48.470122,APPROXIMATE,-1.152420,-48.465938,-1.174713,-48.474167,NaN
4,"[{'long_name': 'Parque Verde', 'short_name': '...","Parque Verde, Belém - PA, Brazil",ChIJ0y8res9hpJIRZYq55xaMzfA,"['political', 'sublocality', 'sublocality_leve...",-1.347171,-48.435147,-1.376068,-48.460912,-1.367325,-48.443752,APPROXIMATE,-1.347171,-48.435147,-1.376068,-48.460912,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,"[{'long_name': 'Maracangalha', 'short_name': '...","Maracangalha, Belém - PA, Brazil",ChIJ0bJXt3-JpJIRga4a1xCwbZA,"['political', 'sublocality', 'sublocality_leve...",-1.394400,-48.470004,-1.408716,-48.492040,-1.399177,-48.481225,APPROXIMATE,-1.394400,-48.470004,-1.408716,-48.492040,NaN
72,"[{'long_name': 'Agulha (Icoaraci)', 'short_nam...","Agulha (Icoaraci), Belém - PA, 66811-033, Brazil",ChIJufLAiKhmpJIRE8_2LpkEBlI,"['political', 'sublocality', 'sublocality_leve...",NaN,NaN,NaN,NaN,-1.307735,-48.468183,APPROXIMATE,-1.296580,-48.452175,-1.318891,-48.484190,NaN
73,"[{'long_name': 'Paracuri 1(Icoaraci)', 'short_...","Paracuri 1(Icoaraci), Belém - PA, Brazil",ChIJfdLRrVdhpJIR7MSpoU4GVA8,"['political', 'sublocality', 'sublocality_leve...",-1.309411,-48.468147,-1.327215,-48.491493,-1.315202,-48.481225,APPROXIMATE,-1.309411,-48.468147,-1.327215,-48.491493,NaN
74,"[{'long_name': 'Marahú', 'short_name': 'Marahú...","Marahú, Belém - PA, Brazil",ChIJH_idhwIWpJIRjAp3bjKoa2Y,"['political', 'sublocality', 'sublocality_leve...",-1.070399,-48.392562,-1.087686,-48.419066,-1.074115,-48.399243,APPROXIMATE,-1.070399,-48.392562,-1.087686,-48.419066,NaN
